In [3]:
import numpy as np
import pandas as pd
import gmaps as gm
import math
import os


In [4]:
#data as of 7/23/23
places = pd.read_csv("NJJ_places_geocodio.csv")


In [5]:
#dropping rows that have no lat-long
acc = places[places['Accuracy Score'] == 0].index
places = places.drop(acc)
places = places.reset_index(drop=True)

In [6]:
#formatting data for gmaps
latlong = places[['Latitude','Longitude']]
places['transaction amount'] = pd.to_numeric(places['transaction amount'])
places['transaction amount'] = places['transaction amount'].div(10)


In [7]:
gm.configure(api_key=os.environ['GOOGLE_API_KEY'])
fig = gm.figure(map_type='ROADMAP')

#creating weighted heatmap
heatmap_layer = gm.heatmap_layer(latlong, weights=places['transaction amount'], max_intensity = 50, point_radius=15)
fig.add_layer(heatmap_layer)


In [8]:
#math for circle around stadium
#need to account for curvature of earth/lat-long coordinate system
center = (40.868837,-74.194292)
circle1 = [(0,0)] * 90
lat_dpm = 360/24859.73 #degrees per mile at this lat long
long_dpm = 360/19039.39
rad_m = 12
for i in range(0,360,4):
    lat = center[0] + math.cos(math.radians(i)) * lat_dpm  * rad_m
    long = center[1] + math.sin(math.radians(i)) * long_dpm  * rad_m
    circle1[i//4] = (lat,long)
    
#ad radius around hinchcliffe stadium    
hinchcliffe = (40.9186296643137, -74.18224257317111)
circle2 = [(0,0)] * 90
rad_m = 4
for i in range(0,360,4):
    lat = hinchcliffe[0] + math.cos(math.radians(i)) * lat_dpm  * rad_m
    long = hinchcliffe[1] + math.sin(math.radians(i)) * long_dpm  * rad_m
    circle2[i//4] = (lat,long)
    
#creating markers and polygon for this map
NJJ = gm.Marker(center,info_box_content="Yogi Berra Stadium")
HCS = gm.Marker(hinchcliffe,info_box_content="Hinchcliffe Stadium")
ad_rad = gm.Polygon(circle1,stroke_color='blue',fill_color='blue')
ad_rad_2 = gm.Polygon(circle2,stroke_color='blue', fill_color='blue')
drawing = gm.drawing_layer(features = [NJJ,HCS,ad_rad,ad_rad_2],show_controls=False)
fig.add_layer(drawing)

In [9]:
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
#testing for intensity and radii
#heatmap_layer.max_intensity = 50
#heatmap_layer.point_radius = 15